In [2]:
#import packages
from __future__ import print_function
import json
from sklearn.preprocessing import MultiLabelBinarizer
import numpy as np
import tensorflow as tf

In [3]:
#load the file
filepath = "student_vectors_n_task_10_n_limit_10000.json"
student_vectors = json.load(open(filepath))

In [4]:
# Preparing a mapping from qual_id and ccssm to one hot representation

# collect all qual_ids, ccssm_labels, task_ids in separate lists
all_qual_ids = []
all_ccssm_labels = []
all_task_ids = []
for i in student_vectors:
    for j in student_vectors[i]:
        all_qual_ids.append(j['qual_id'])
        all_ccssm_labels.append(j['ccssm'])
        all_task_ids.append(j['task_id'])

# make a set of unique values from the above lists
unique_ids = set(all_qual_ids)
unique_labels = set(all_ccssm_labels)
unique_tasks = set(all_task_ids)
# print("Number of unique labels in this dataset " + str(len(unique_labels))) #unique labels
# print("Number of unique lessons/tasks in this dataset "+str(len(unique_tasks))) #unique lessons
# print("Number of unique questions in this dataset "+str(len(unique_ids))) #this is the length of bit vector (number of unique qual_ids)

# generate vectors to give to fit_transform in multilabelbinarizer to further generate unique 1-hot encoding
transform_ids = []
for i in unique_ids:
    transform_ids.append([i])

transform_labels = []
for i in unique_labels:
    transform_labels.append([i])
    
# generate dictionary that maps labels and qual_ids to their respective 1-hot encoding
enc = MultiLabelBinarizer()
qual_ids_1hot = (enc.fit_transform(transform_ids)).astype(float)
qual_ids_classes = enc.classes_
qual_ids_dict = dict(zip(unique_ids, qual_ids_1hot))
labels_1hot = enc.fit_transform(transform_labels).astype(float)
labels_classes = enc.classes_
labels_dict = dict(zip(unique_labels,labels_1hot))

In [5]:
#preparing a mapping from qual_id and ccssm to normalized frequency distribution
freq_dist = {}
total_occ = {}
total_interactions = 0
for i in student_vectors:
    for j in student_vectors[i]:
        total_interactions += 1
        if j['qual_id'] in total_occ:
            total_occ[j['qual_id']] += 1
        else:
            total_occ[j['qual_id']] = 1
        if j['ccssm'] in total_occ:
            total_occ[j['ccssm']] += 1
        else:
            total_occ[j['ccssm']] = 1
        if j['correct'] == True:
            if j['qual_id'] in freq_dist:
                freq_dist[j['qual_id']] += 1
            else:
                freq_dist[j['qual_id']] = 1
            if j['ccssm'] in freq_dist:
                freq_dist[j['ccssm']] += 1
            else:
                freq_dist[j['ccssm']] = 1
        if j['correct'] == False: #what if all occurences were answered incorrectly
            if j['qual_id'] not in freq_dist:
                freq_dist[j['qual_id']] = 0
            if j['ccssm'] not in freq_dist:
                freq_dist[j['ccssm']] = 0
# print(freq_dist)
# print("\n\n\n")
# print(total_occ)
# for i in freq_dist:
#     if freq_dist[i] != total_occ[i]:
#         print(i)
#         print(freq_dist[i])
#         print(total_occ[i])
#         print("\n")
for i in freq_dist:
    freq_dist[i] = float(freq_dist[i]) / float(total_occ[i])
    
print(total_interactions)

52209


In [6]:
#construct a vector where we place the frequency of a qual_id/ccssm at the position of the qual_id/ccssm in the vector.
y_vector = np.zeros([len(unique_labels) + len(unique_ids)])

for i in student_vectors:
    for j in student_vectors[i]:
        id_pos = np.argmax(qual_ids_dict[j['qual_id']])
        label_pos = np.argmax(labels_dict[j['ccssm']]) + len(unique_ids)
        if(y_vector[id_pos] == 0.0):
            y_vector[id_pos] = freq_dist[j['qual_id']]
        if(y_vector[label_pos] == 0.0):
            y_vector[label_pos] = freq_dist[j['ccssm']]

In [10]:
# prepare labels array of size: [total_interactions, len(unique_labels) + len(unique_ids)]
# but total_interactions should not take into account 
# first counting cases with effective interaction
index = 0
flag = False
for i in student_vectors:
    for j in student_vectors[i]:
        if(j['correct'] == True):
            #correct in first try
            index += 1
        elif(j['correct'] == False and j['second_try'] == True):
            #false even after second try
            index += 1
print(index)
effective_interactions = index
y_tensor = np.zeros([effective_interactions, len(unique_labels) + len(unique_ids)])

index = 0
flag = False
for i in student_vectors:
    for j in student_vectors[i]:
        if(j['correct'] == True):
            #correct in first try
            a = qual_ids_dict[j['qual_id']]
            b = labels_dict[j['ccssm']]
            temp = np.append(a, b)
            y_tensor[index] = temp
            index += 1
        elif(j['correct'] == False and j['second_try'] == True):
            a = qual_ids_dict[j['qual_id']]
            b = labels_dict[j['ccssm']]
            temp = np.append(a, b)
            #temp = temp * (1.0/3.0)
            temp = temp * (0.0)
            y_tensor[index] = temp
            index += 1

# prepare predictions array of size: [effective_interactions, len(unique_labels) + len(unique_ids)]; 
pred_tensor = np.zeros([effective_interactions, len(unique_labels) + len(unique_ids)])
for i in range(effective_interactions):
    pred_tensor[i] = y_vector
    
#process y_tensor for auc
for i in range(len(y_tensor)):
    for j in range(len(y_tensor[0])):
        if(y_tensor[i][j] != 0.0 and y_tensor[i][j] != 1.0):
            print("hahaha")
            break

44758


In [11]:
# pred = tf.placeholder(tf.float32, [None, len(unique_labels) + len(unique_ids)])
# y = tf.placeholder(tf.float32, [None, len(unique_labels) + len(unique_ids)])
# if tf.VERSION == '1.3.0':
#     auc, opts = tf.metrics.auc(labels = y, predictions = pred, curve='ROC')
# elif tf.VERSION == '0.12.1': #summit's tensorflow version API doc: https://www.tensorflow.org/versions/r0.12/api_docs/
#     auc, opts = tf.contrib.metrics.streaming_auc(labels = y, predictions = pred, curve='ROC')
# with tf.Session() as sess:
#     sess.run(tf.global_variables_initializer())
#     sess.run(tf.local_variables_initializer()) #https://github.com/tensorflow/tensorflow/issues/3971
#     auc_out,opts_out = sess.run([auc,opts],feed_dict={y: y_tensor, pred: pred_tensor})
#     print(auc_out)
#     print(opts_out)

In [12]:
import numpy as np
from sklearn.metrics import roc_auc_score
y_true = y_tensor
y_scores = pred_tensor
print("sklearn auc: {}".format(roc_auc_score(y_true, y_scores)))


ValueError: Only one class present in y_true. ROC AUC score is not defined in that case.